In [92]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [93]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [94]:
df = pd.read_csv("/content/gdrive/MyDrive/Datasets/dataset.csv")
df = df.dropna(axis = 0)
df.shape

(28015, 2)

In [95]:
df.head()

,category,text
0,Благоустройство территории,На трассе Нижнекамск-Чистополь в лесополосе у ...
1,Благоустройство территории,Во дворе дома Ленинградская 29 отсутствуют пар...
2,Содержание и ремонт муниципальных дорог,Добрый день! Не осуществляется уборка снега в ...
3,Благоустройство территории,Добрый день! Не осуществляется вывоз мусора с ...
4,Содержание и ремонт муниципальных дорог,на данном участке проезжей части от ул.Халитов...


In [96]:
df.text[200]

'Около детского сада месиво, ребенка невозможно провезти даже на салазках'

In [97]:
df['text'][10]

'Не убирают снег на тротуаре. Точнее при уборке дороги, весь снег с дороги расположили на тротуаре. Нужно очистить тротуар, для передвижения пешеходов.'

In [98]:
df['category'].nunique()

36

In [99]:
df['category'].value_counts()

Благоустройство территории                                                              12967
Содержание и ремонт муниципальных дорог                                                  6948
Нарушение в наружной рекламе                                                             2079
Организация дорожного движения                                                           2040
Общественный транспорт                                                                    936
Бездомные животные                                                                        726
Поликлиники и больницы                                                                    524
Парки и скверы                                                                            406
Cвалки                                                                                    335
Мобильная связь                                                                           200
Воздух                                                      

In [100]:
q1 = df.category.value_counts()[df.category.value_counts() < 25].index

In [101]:
df.category = df.category.replace(q1, 'Разное')

In [102]:
df['category'].value_counts()

Благоустройство территории                                 12967
Содержание и ремонт муниципальных дорог                     6948
Нарушение в наружной рекламе                                2079
Организация дорожного движения                              2040
Общественный транспорт                                       936
Бездомные животные                                           726
Поликлиники и больницы                                       524
Парки и скверы                                               406
Cвалки                                                       335
Мобильная связь                                              200
Воздух                                                       142
Капитальный ремонт                                           127
Детские сады                                                 126
Разное                                                       119
Санитарное состояние                                          91
Доступная среда          

# Предварительная обработка текста

In [103]:
!pip install pymorphy2

In [104]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Удаление стоп-слов

In [105]:
all_stopwords = stopwords.words("russian")

In [106]:
q2 = "Саша и Маша"

In [107]:
list1 = []
for i in q2.split(" "):
  if i not in all_stopwords:
    list1.append(i)

In [108]:
(" ").join(list1)

'Саша Маша'

In [109]:
%%time
list2 = []
for j in df.text:
  list1 = []
  for i in j.split(" "):
    if i not in all_stopwords:
      list1.append(i)
  
  list2.append((" ").join(list1))

CPU times: user 1.12 s, sys: 13.9 ms, total: 1.13 s
Wall time: 1.14 s


In [110]:
df.text = list2

Приведение к нижнему регистру и удаление пунктуации

In [111]:
import re

In [112]:
%%time
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
pattern11 = re.compile(r'\s+')

list2 = []
for j in df.text:
  buff = j.lower()
  buff = re.sub(patterns, '', buff)
  buff = re.sub(pattern11, ' ', buff)
  list2.append(buff)

CPU times: user 367 ms, sys: 5.96 ms, total: 373 ms
Wall time: 374 ms


In [113]:
df.text = list2

In [114]:
df.text[:5]

0    на трассе нижнекамскчистополь лесополосе пгт к...
1    во дворе дома ленинградская отсутствуют парков...
2    добрый день не осуществляется уборка снега цен...
3    добрый день не осуществляется вывоз мусора тер...
4    данном участке проезжей части улхалитова дома ...
Name: text, dtype: object

# Остальное

In [115]:
from nltk.stem.snowball import SnowballStemmer

In [116]:
stemmer = SnowballStemmer("russian")
def stm(doc):
    tokens = []
    for token in doc.split():
      token = stemmer.stem(token)
      tokens.append(token)
    if len(tokens) > 0:
        return (" ").join(tokens)
    return None

In [117]:
df.text = df.text.apply(stm)

# Векторизация

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [119]:
df.head()

,category,text
0,Благоустройство территории,на трасс нижнекамскчистопол лесополос пгт камс...
1,Благоустройство территории,во двор дом ленинградск отсутств парковочн мес...
2,Содержание и ремонт муниципальных дорог,добр ден не осуществля уборк снег центр город ...
3,Благоустройство территории,добр ден не осуществля вывоз мусор территор жи...
4,Содержание и ремонт муниципальных дорог,дан участк проезж част улхалитов дом № улхалит...


In [120]:
X = df.text
y = df.category

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = y)
X_train = X_train.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [122]:
vectorizer = TfidfVectorizer(max_features = 10000)

In [123]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [124]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score

In [125]:
%%time
model_SVC = SVC(C = 1, kernel = 'linear')
model_SVC.fit(X_train, y_train)
res = model_SVC.predict(X_test)

CPU times: user 56.2 s, sys: 308 ms, total: 56.5 s
Wall time: 56.4 s


In [126]:
f1_svm = f1_score(y_test, res, average='macro')
f1_svm

0.591859054448229

In [127]:
print(classification_report(y_test, res))

                                                         precision    recall  f1-score   support

                                                 Cвалки       0.62      0.36      0.45        67
                                     Бездомные животные       0.97      0.94      0.95       145
                             Благоустройство территории       0.80      0.85      0.83      2593
                                                   Вода       0.00      0.00      0.00         5
                                                 Воздух       1.00      0.83      0.91        29
                                           Детские сады       0.88      0.28      0.42        25
                                        Доступная среда       0.88      0.44      0.58        16
                                 Жилищное строительство       0.00      0.00      0.00         7
                                     Капитальный ремонт       0.79      0.44      0.56        25
                             

In [128]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
%%time
model_SVC = RandomForestClassifier()
model_SVC.fit(X_train, y_train)
res = model_SVC.predict(X_test)

CPU times: user 41.9 s, sys: 82.7 ms, total: 42 s
Wall time: 41.9 s


In [130]:
f1_rf = f1_score(y_test, res, average='macro')
f1_rf

0.5268914750612422

In [131]:
from sklearn.neighbors import KNeighborsClassifier

In [132]:
%%time
model_SVC = KNeighborsClassifier()
model_SVC.fit(X_train, y_train)
res = model_SVC.predict(X_test)

CPU times: user 6.5 s, sys: 594 ms, total: 7.1 s
Wall time: 7.06 s


In [136]:
def getFeedbackMessage(requestCategory):
  message = 'Спасибо. '
  if requestCategory == 'Благоустройство территории':
    message = message + 'Ваше обращение отправлено в департамент благоустройства.'
  if requestCategory == 'Содержание и ремонт муниципальных дорог':
    message = message + 'Ваше обращение отправлено в департамент дорожного строительства.'
  return message

In [137]:
while True:
  request = input('Ваше сообщение > ')
  if request == 'спасибо':
    print('Благодарим за обращение!')
    break
  classificationResult = model_SVC.predict(vectorizer.transform([request]))
  print(getFeedbackMessage(classificationResult))

Ваше сообщение > мусор на улице
Спасибо. Ваше обращение отправлено в департамент благоустройства.
Ваше сообщение > спасибо
Благодарим за обращение!
